In [1]:
using PCT

[ Info: Precompiling PCT [ef708a43-c8a3-43f4-8f65-1f04ee4c5bb0]


In [3]:
@profview h, _ = @pct pullback((z::C) -> z' * z)

LoadError: LoadError: UndefVarError: `@profview` not defined
in expression starting at In[3]:2

In [4]:
println(verbose(h))

Pullback((z::C->
    (*
        z::C,
        z::C'
    )::C
::[C->C])::[C×C->C]


In [5]:
reduce_pullback(h)[1]

(z, _K) -> 
    ((z, _K) -> 
        (((z, _K) -> 
            _K)(z, (_K⋅z))+((z, _K) -> 
            ((z, _K) -> 
                _K)(z, _K'))(z, (_K⋅z'))))(z, _K)

In [9]:
p = eval_all(reduce_pullback(h)[1])
q = eval_all(call(p, first(ff(p)), constant(1)))

(z⋅2.0)

In [10]:
simplify(q) |> first

  0.000000 seconds


(z⋅2.0)

In [2]:
f, ctx = @pct begin
    @space H begin
        type=(I, I)->C
        symmetries=(((2,1), :conj),)
    end
    
    @space V begin
        type=(I,)->C
    end

    @space T begin
        type=(I,I,I,I) -> C
        symmetries=(((2,1,4,3), :conj), ((3,4,1,2), :id))
    end
                
    @space U begin
        type=(I,I)->C
    end
                    
    (A::H, J::T) -> _
end

(Map(MapType(VecType(AbstractPCTType[MapType(VecType(AbstractPCTType[I(), I()]), C(), Dict{Symbol, Any}(:name => :H, :symmetries => (((2, 1), :conj),))), MapType(VecType(AbstractPCTType[I(), I(), I(), I()]), C(), Dict{Symbol, Any}(:name => :T, :symmetries => (((2, 1, 4, 3), :conj), ((3, 4, 1, 2), :id))))]), UndeterminedPCTType(), Dict{Any, Any}()), PCTVector(VecType(AbstractPCTType[MapType(VecType(AbstractPCTType[I(), I()]), C(), Dict{Symbol, Any}(:name => :H, :symmetries => (((2, 1), :conj),))), MapType(VecType(AbstractPCTType[I(), I(), I(), I()]), C(), Dict{Symbol, Any}(:name => :T, :symmetries => (((2, 1, 4, 3), :conj), ((3, 4, 1, 2), :id))))]), Var{MapType}[Var{MapType}(MapType(VecType(AbstractPCTType[I(), I()]), C(), Dict{Symbol, Any}(:name => :H, :symmetries => (((2, 1), :conj),))), :A), Var{MapType}(MapType(VecType(AbstractPCTType[I(), I(), I(), I()]), C(), Dict{Symbol, Any}(:name => :T, :symmetries => (((2, 1, 4, 3), :conj), ((3, 4, 1, 2), :id)))), :J)]), Var{UndeterminedPCTTyp

In [11]:
f

(A, J) -> 
    _

In [39]:
g = @pct f ctx pullback((x::V) -> sum((i, j), x(i)' * A(i, j) * x(j)))

(A, J) -> 
    𝒫((x) -> 
        ∑((i_0, i_1), (x(i_0)⋅A(i_1, i_0)⋅x(i_1)')))

In [40]:
println(verbose(g))

(A::[I×I->C], J::[I×I×I×I->C]->
    Pullback((x::[I->C]->
        ∑((i_0::I, i_1::I),
            (*
                x::[I->C](i_0::I)::C,
                A::[I×I->C](i_1::I, i_0::I)::C,
                x::[I->C](i_1::I)::C'
            )::C
        )::C
    ::[[I->C]->C])::[[I->C]×C->[I->C]]
::[[I×I->C]×[I×I×I×I->C]->[[I->C]×C->[I->C]]]


In [41]:
p = fc(eval_all(reduce_pullback(g)))

(x, _K) -> 
    (i_2) -> 
        ∑((i_0, i_1), (δ(i_2, i_1, (x(i_0)⋅A(i_1, i_0)⋅_K'))+δ(i_2, i_0, (x(i_1)⋅A(i_1, i_0)'⋅_K))))

In [42]:
q = eval_all(call(p, first(ff(p)), constant(1)))

(i_2) -> 
    ∑((i_0, i_1), (δ(i_2, i_1, (x(i_0)⋅A(i_1, i_0)))+δ(i_2, i_0, (x(i_1)⋅A(i_1, i_0)'))))

In [43]:
simplify(q) |> first

  0.000019 seconds (7 allocations: 304 bytes)
1 sum_dist
∑((i_0, i_1), (δ(i_2, i_1, (x(i_0)⋅A(i_1, i_0)))+δ(i_2, i_0, (x(i_1)⋅A(i_1, i_0)'))))
(∑((i_0, i_1), δ(i_2, i_0, (x(i_1)⋅A(i_1, i_0)')))+∑((i_0, i_1), δ(i_2, i_0, (x(i_1)⋅A(i_1, i_0)'))))

  0.000281 seconds (293 allocations: 13.062 KiB)
2 contract_delta
(∑((i_0, i_1), δ(i_2, i_0, (x(i_1)⋅A(i_1, i_0)')))+∑((i_0, i_1), δ(i_2, i_0, (x(i_1)⋅A(i_1, i_0)'))))
(∑((i_0), (x(i_0)⋅A(i_2, i_0)))+∑((i_0, i_1), δ(i_2, i_0, (x(i_1)⋅A(i_1, i_0)'))))
directed!

  0.000041 seconds (215 allocations: 9.688 KiB)
3 contract_delta
(∑((i_0), (x(i_0)⋅A(i_2, i_0)))+∑((i_0, i_1), δ(i_2, i_0, (x(i_1)⋅A(i_1, i_0)'))))
(∑((i_0), (x(i_0)⋅A(i_2, i_0)))+∑((i_0), (x(i_0)⋅A(i_2, i_0))))
directed!

  0.000019 seconds (37 allocations: 1.625 KiB)
4 gcd
(∑((i_0), (x(i_0)⋅A(i_2, i_0)))+∑((i_0), (x(i_0)⋅A(i_2, i_0))))
((1+1)⋅∑((i_0), (x(i_0)⋅A(i_2, i_0))))

  0.000042 seconds (244 allocations: 11.312 KiB)
5 add_const
((1+1)⋅∑((i_0), (x(i_0)⋅A(i_2, i_0))))
(∑((i_0), (x

(i_2) -> 
    (∑((i_0), (x(i_0)⋅A(i_2, i_0)))⋅2)

In [44]:
g = @pct f ctx pullback((C::U) -> sum((i,j,p,q,r,s), C(p,i)' * C(q,i) * C(r,j)' * C(s,j) * J(p,q,r,s)))

(A, J) -> 
    𝒫((C) -> 
        ∑((i_0, i_1, i_2, i_3, i_4, i_5), (C(i_1, i_5)⋅C(i_3, i_4)⋅J(i_1, i_0, i_3, i_2)'⋅C(i_2, i_4)'⋅C(i_0, i_5)')))

In [45]:
p = fc(eval_all(reduce_pullback(g)))

(C, _K) -> 
    (i_6, i_7) -> 
        ∑((i_0, i_1, i_2, i_3, i_4, i_5), (δ(i_7, i_5, δ(i_6, i_2, (J(i_1, i_4, i_2, i_3)⋅C(i_4, i_0)⋅C(i_3, i_5)⋅C(i_1, i_0)'⋅_K)))+δ(i_7, i_5, δ(i_6, i_3, (C(i_1, i_0)⋅C(i_2, i_5)⋅J(i_1, i_4, i_2, i_3)'⋅C(i_4, i_0)'⋅_K')))+δ(i_7, i_0, δ(i_6, i_1, (J(i_1, i_4, i_2, i_3)⋅C(i_4, i_0)⋅C(i_3, i_5)⋅C(i_2, i_5)'⋅_K)))+δ(i_7, i_0, δ(i_6, i_4, (C(i_1, i_0)⋅C(i_2, i_5)⋅J(i_1, i_4, i_2, i_3)'⋅C(i_3, i_5)'⋅_K')))))

In [46]:
q = eval_all(call(p, first(ff(p)), constant(1)))

(i_6, i_7) -> 
    ∑((i_0, i_1, i_2, i_3, i_4, i_5), (δ(i_7, i_5, δ(i_6, i_2, (J(i_1, i_4, i_2, i_3)⋅C(i_4, i_0)⋅C(i_3, i_5)⋅C(i_1, i_0)')))+δ(i_7, i_5, δ(i_6, i_3, (C(i_1, i_0)⋅C(i_2, i_5)⋅J(i_1, i_4, i_2, i_3)'⋅C(i_4, i_0)')))+δ(i_7, i_0, δ(i_6, i_1, (J(i_1, i_4, i_2, i_3)⋅C(i_4, i_0)⋅C(i_3, i_5)⋅C(i_2, i_5)')))+δ(i_7, i_0, δ(i_6, i_4, (C(i_1, i_0)⋅C(i_2, i_5)⋅J(i_1, i_4, i_2, i_3)'⋅C(i_3, i_5)')))))

In [8]:
Profile.clear()
@profile p==p
pprof()

"profile.pb.gz"

Main binary filename not available.
Serving web UI on http://localhost:57599


In [ ]:
(A::H) -> (x::V) -> x A x

In [67]:
f, ctx = @pct begin 

    @domain P begin
        base = I
        lower = -N
        upper = N-1
        periodic = true
    end

    @domain Q begin
        base = I
        lower = -N
        upper = N
        contractable = false
    end

    @space Mmn begin
        type = (I, I, I, I) -> C
        symmetries = (((2, 1, 4, 3), :conj), )
    end

    @space Sym begin
        type = (I, ) -> C
        symmetries = (((1, ), :ineg), )
    end

    @space Gauge begin
        type = (I, I, I) -> C
    end

    @space Density begin
        type = (I, I) -> C
    end

    #= @space SymC begin
        type = (I, ) -> C
        symmetries = (((1, ), :inegc), )
    end =#

    (S::Mmn, w::Sym) -> _
end

g = @pct f ctx pullback((U::Gauge) -> ((ρ::Density) -> sum((n::I, b::Q), ρ(n, b)' * ρ(n, b)))(
    (n::I, b::Q) -> sum((k::P, p, q), U(p, n, k)' * S(p,q,k,k+b) * U(q, n, k+b))))
eval_all(g)
g_1 = fc(eval_all(reduce_pullback(eval_all(g))))
g_2 = eval_all(call(g_1, first(ff(g_1)), constant(1)))

(i_0, i_1, i_2) -> 
    ∑((n, b), (∑((k, p, q), (δ(i_2, (b+k), δ(i_1, n, δ(i_0, q, (U(p, n, k)⋅∑((k, p, q), (U(q, n, (b+k))⋅S(p, q, k, (b+k))⋅U(p, n, k)'))⋅S(p, q, k, (b+k))'))))+δ(i_2, k, δ(i_1, n, δ(i_0, p, (U(q, n, (b+k))⋅S(p, q, k, (b+k))⋅∑((k, p, q), (U(p, n, k)⋅U(q, n, (b+k))'⋅S(p, q, k, (b+k))'))))))))⋅2.0))

In [69]:
g_3 = simplify(g_2; settings=Dict(:clench_sum=>true)) |> first

  0.023775 seconds (337.30 k allocations: 10.832 MiB)
1 sum_out
∑((n, b), (∑((k, p, q), (δ(i_2, (b+k), δ(i_1, n, δ(i_0, q, (U(p, n, k)⋅∑((k, p, q), (U(q, n, (b+k))⋅S(p, q, k, (b+k))⋅U(p, n, k)'))⋅S(p, q, k, (b+k))'))))+δ(i_2, k, δ(i_1, n, δ(i_0, p, (U(q, n, (b+k))⋅S(p, q, k, (b+k))⋅∑((k, p, q), (U(p, n, k)⋅U(q, n, (b+k))'⋅S(p, q, k, (b+k))'))))))))⋅2.0))
∑((b), (∑((n, k, p, q), (δ(i_2, (b+k), δ(i_1, n, δ(i_0, q, (U(p, n, k)⋅∑((k, p, q), (U(q, n, (b+k))⋅S(p, q, k, (b+k))⋅U(p, n, k)'))⋅S(p, q, k, (b+k))'))))+δ(i_2, k, δ(i_1, n, δ(i_0, p, (U(q, n, (b+k))⋅S(p, q, k, (b+k))⋅∑((k, p, q), (U(p, n, k)⋅U(q, n, (b+k))'⋅S(p, q, k, (b+k))'))))))))⋅2.0))
yesdirected!

  0.038001 seconds (493.02 k allocations: 16.019 MiB)
2 sum_out
∑((b), (∑((n, k, p, q), (δ(i_2, (b+k), δ(i_1, n, δ(i_0, q, (U(p, n, k)⋅∑((k, p, q), (U(q, n, (b+k))⋅S(p, q, k, (b+k))⋅U(p, n, k)'))⋅S(p, q, k, (b+k))'))))+δ(i_2, k, δ(i_1, n, δ(i_0, p, (U(q, n, (b+k))⋅S(p, q, k, (b+k))⋅∑((k, p, q), (U(p, n, k)⋅U(q, n, (b+k))'⋅S(p, q, k, (

(i_0, i_1, i_2) -> 
    (2.0⋅(∑((b, q), (U(q, i_1, (b+i_2))⋅S(i_0, q, i_2, (b+i_2))⋅∑((k, q), (∑((p), (U(p, i_1, k)⋅S(p, q, k, (b+k))'))⋅U(q, i_1, (b+k))'))))+∑((b, p), (U(p, i_1, ((-1.0⋅b)+i_2))⋅∑((k, q), (U(q, i_1, (b+k))⋅∑((p), (S(p, q, k, (b+k))⋅U(p, i_1, k)'))))⋅S(p, i_0, ((-1.0⋅b)+i_2), i_2)'))))

In [34]:
using Profile
using Pkg
Pkg.add("PProf")
using PProf

   Resolving package versions...
   Installed XML2_jll ────────────────── v2.12.0+0
   Installed Glib_jll ────────────────── v2.76.5+0
   Installed Xorg_xtrans_jll ─────────── v1.5.0+0
   Installed Libiconv_jll ────────────── v1.17.0+0
   Installed Pango_jll ───────────────── v1.50.14+0
   Installed ProgressMeter ───────────── v1.9.0
   Installed Xorg_libpthread_stubs_jll ─ v0.1.1+0
   Installed ProtoBuf ────────────────── v1.0.14
   Installed Xorg_libX11_jll ─────────── v1.8.6+0
   Installed BufferedStreams ─────────── v1.2.1
   Installed PProf ───────────────────── v3.1.0
   Installed FreeType2_jll ───────────── v2.13.1+0
   Installed Xorg_libXau_jll ─────────── v1.0.11+0
   Installed Xorg_libXdmcp_jll ───────── v1.1.4+0
   Installed Xorg_libxcb_jll ─────────── v1.15.0+0
   Installed CodecZlib ───────────────── v0.7.3
    Updating `~/kaer_morhen/projects/QLD/PCT/Project.toml`
  [e4faabce] + PProf v3.1.0
    Updating `~/kaer_morhen/projects/QLD/PCT/Manifest.toml`
  [1520ce14] + Abstra

┌ Warning: Module JLLWrappers with build ID fafbfcfd-cb8c-49c8-0005-da57d32b1727 is missing from the cache.
│ This may mean JLLWrappers [692b3bcd-3c85-4b1f-b108-f13ce0eb3210] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1793
[ Info: Skipping precompilation since __precompile__(false). Importing pprof_jll [cf2c5f97-e748-59fa-a03f-dda3c62118cb].
[ Info: Precompiling Graphviz_jll [3c863552-8265-54e4-a6dc-903eb78fde85]
┌ Warning: Module JLLWrappers with build ID fafbfcfd-cb8c-49c8-0005-da57d32b1727 is missing from the cache.
│ This may mean JLLWrappers [692b3bcd-3c85-4b1f-b108-f13ce0eb3210] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1793
[ Info: Skipping precompilation since __precompile__(false). Importing Graphviz_jll [3c863552-8265-54e4-a6dc-903eb78fde85].
[ Info: Precompiling Cairo_jll [83423d85-b0ee-5818-9007-b63ccbeb887a]
┌ Warning: Module JLLWrappers with build ID fafbfcfd-cb8c-49c8-0005-da

┌ Warning: Module JLLWrappers with build ID fafbfcfd-cb8c-49c8-0005-da57d32b1727 is missing from the cache.
│ This may mean JLLWrappers [692b3bcd-3c85-4b1f-b108-f13ce0eb3210] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1793
[ Info: Skipping precompilation since __precompile__(false). Importing Libuuid_jll [38a345b3-de98-5d2b-a5d3-14cd9215e700].
[ Info: Precompiling Expat_jll [2e619515-83b5-522b-bb60-26c02a35a201]
┌ Warning: Module JLLWrappers with build ID fafbfcfd-cb8c-49c8-0005-da57d32b1727 is missing from the cache.
│ This may mean JLLWrappers [692b3bcd-3c85-4b1f-b108-f13ce0eb3210] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1793
[ Info: Skipping precompilation since __precompile__(false). Importing Expat_jll [2e619515-83b5-522b-bb60-26c02a35a201].
[ Info: Precompiling LZO_jll [dd4b983a-f0e5-5f8d-a1b7-129d4a5fb1ac]
┌ Warning: Module JLLWrappers with build ID fafbfcfd-cb8c-49c8-0005-da57d32b

In [70]:
g_4 = simplify(g_3; settings=Dict(:symmetry=>true)) |> first

  0.010030 seconds (115.85 k allocations: 3.589 MiB)
1 sum_in
(2.0⋅(∑((b, q), (U(q, i_1, (b+i_2))⋅S(i_0, q, i_2, (b+i_2))⋅∑((k, q), (∑((p), (U(p, i_1, k)⋅S(p, q, k, (b+k))'))⋅U(q, i_1, (b+k))'))))+∑((b, p), (U(p, i_1, ((-1.0⋅b)+i_2))⋅∑((k, q), (U(q, i_1, (b+k))⋅∑((p), (S(p, q, k, (b+k))⋅U(p, i_1, k)'))))⋅S(p, i_0, ((-1.0⋅b)+i_2), i_2)'))))
(2.0⋅(∑((b, p), (U(p, i_1, ((-1.0⋅b)+i_2))⋅∑((k, q), (U(q, i_1, (b+k))⋅∑((p), (S(p, q, k, (b+k))⋅U(p, i_1, k)'))))⋅S(p, i_0, ((-1.0⋅b)+i_2), i_2)'))+∑((b, q, k, i_3), (U(q, i_1, (b+i_2))⋅S(i_0, q, i_2, (b+i_2))⋅∑((p), (U(p, i_1, k)⋅S(p, i_3, k, (b+k))'))⋅U(i_3, i_1, (b+k))'))))
yesdirected!

  0.010254 seconds (160.01 k allocations: 4.776 MiB)
2 sum_in
(2.0⋅(∑((b, p), (U(p, i_1, ((-1.0⋅b)+i_2))⋅∑((k, q), (U(q, i_1, (b+k))⋅∑((p), (S(p, q, k, (b+k))⋅U(p, i_1, k)'))))⋅S(p, i_0, ((-1.0⋅b)+i_2), i_2)'))+∑((b, q, k, i_3), (U(q, i_1, (b+i_2))⋅S(i_0, q, i_2, (b+i_2))⋅∑((p), (U(p, i_1, k)⋅S(p, i_3, k, (b+k))'))⋅U(i_3, i_1, (b+k))'))))
(2.0⋅(∑((b, q, k, i_3), 

  0.062319 seconds (824.06 k allocations: 24.389 MiB, 15.23% gc time)
14 sum_sym
(2.0⋅(∑((b, q, k, i_3, p), (U(q, i_1, (b+i_2))⋅U(p, i_1, ((-1.0⋅b)+k))⋅S(i_3, p, k, ((-1.0⋅b)+k))⋅U(i_3, i_1, k)'⋅S(q, i_0, (b+i_2), i_2)'))+∑((b, p, k, q, i_3), (U(q, i_1, (b+k))⋅U(p, i_1, ((-1.0⋅b)+i_2))⋅S(i_3, q, k, (b+k))⋅U(i_3, i_1, k)'⋅S(p, i_0, ((-1.0⋅b)+i_2), i_2)'))))
(∑((b, p, k, q, i_3), (U(q, i_1, (b+k))⋅U(p, i_1, ((-1.0⋅b)+i_2))⋅S(i_3, q, k, (b+k))⋅U(i_3, i_1, k)'⋅S(p, i_0, ((-1.0⋅b)+i_2), i_2)'))⋅4.0)
notdirected

  0.005232 seconds (79.41 k allocations: 2.455 MiB)
15 sum_in
(∑((b, p, k, q, i_3), (U(q, i_1, (b+k))⋅U(p, i_1, ((-1.0⋅b)+i_2))⋅S(i_3, q, k, (b+k))⋅U(i_3, i_1, k)'⋅S(p, i_0, ((-1.0⋅b)+i_2), i_2)'))⋅4.0)
∑((b, p, k, q, i_3), (U(q, i_1, (b+k))⋅U(p, i_1, ((-1.0⋅b)+i_2))⋅S(i_3, q, k, (b+k))⋅4.0⋅U(i_3, i_1, k)'⋅S(p, i_0, ((-1.0⋅b)+i_2), i_2)'))
yesdirected!

  0.003225 seconds (39.87 k allocations: 1.575 MiB, 10.83% compilation time)
16 shift
∑((b, p, k, q, i_3), (U(q, i_1, (b+k))⋅U(p, i

  0.043629 seconds (541.40 k allocations: 17.714 MiB)
38 sum_sym
∑((b, p, k, q, i_3), (U(q, i_1, ((-1.0⋅b)+k))⋅U(p, i_1, (b+i_2))⋅4.0⋅U(i_3, i_1, k)'⋅S(q, i_3, ((-1.0⋅b)+k), k)'⋅S(p, i_0, (b+i_2), i_2)'))
∑((b, p, k, q, i_3), (U(q, i_1, (b+k))⋅U(p, i_1, ((-1.0⋅b)+i_2))⋅4.0⋅U(i_3, i_1, k)'⋅S(q, i_3, (b+k), k)'⋅S(p, i_0, ((-1.0⋅b)+i_2), i_2)'))
notdirected

  0.064004 seconds (593.71 k allocations: 19.878 MiB, 15.01% gc time)
39 symmetry
∑((b, p, k, q, i_3), (U(q, i_1, ((-1.0⋅b)+k))⋅U(p, i_1, (b+i_2))⋅4.0⋅U(i_3, i_1, k)'⋅S(q, i_3, ((-1.0⋅b)+k), k)'⋅S(p, i_0, (b+i_2), i_2)'))
∑((b, p, k, q, i_3), (U(q, i_1, ((-1.0⋅b)+k))⋅U(p, i_1, (b+i_2))⋅S(i_0, p, i_2, (b+i_2))⋅4.0⋅U(i_3, i_1, k)'⋅S(q, i_3, ((-1.0⋅b)+k), k)'))
notdirected

  0.043811 seconds (533.38 k allocations: 17.523 MiB)
40 symmetry
∑((b, p, k, q, i_3), (U(q, i_1, ((-1.0⋅b)+k))⋅U(p, i_1, (b+i_2))⋅S(i_3, q, k, ((-1.0⋅b)+k))⋅4.0⋅U(i_3, i_1, k)'⋅S(p, i_0, (b+i_2), i_2)'))
∑((b, p, k, q, i_3), (U(q, i_1, ((-1.0⋅b)+k))⋅U(p, i_1, (b+i_2)

(i_0, i_1, i_2) -> 
    ∑((b, p, k, q, i_3), (U(q, i_1, k)⋅U(p, i_1, (b+i_2))⋅S(i_3, q, (b+k), k)⋅S(i_0, p, i_2, (b+i_2))⋅4.0⋅U(i_3, i_1, (b+k))'))

In [71]:
g_5 = simplify(g_4; settings=Dict(:clench_sum=>true)) |> first

  0.000039 seconds (186 allocations: 7.266 KiB)
1 sum_out
∑((b, p, k, q, i_3), (U(q, i_1, k)⋅U(p, i_1, (b+i_2))⋅S(i_3, q, (b+k), k)⋅S(i_0, p, i_2, (b+i_2))⋅4.0⋅U(i_3, i_1, (b+k))'))
∑((p, k, q, i_3), (U(q, i_1, k)⋅∑((b), (U(p, i_1, (b+i_2))⋅S(i_3, q, (b+k), k)⋅S(i_0, p, i_2, (b+i_2))⋅U(i_3, i_1, (b+k))'))⋅4.0))
yesdirected!

  0.000053 seconds (293 allocations: 11.453 KiB)
2 sum_out
∑((p, k, q, i_3), (U(q, i_1, k)⋅∑((b), (U(p, i_1, (b+i_2))⋅S(i_3, q, (b+k), k)⋅S(i_0, p, i_2, (b+i_2))⋅U(i_3, i_1, (b+k))'))⋅4.0))
∑((k, q, i_3), (U(q, i_1, k)⋅∑((p, b), (U(p, i_1, (b+i_2))⋅S(i_3, q, (b+k), k)⋅S(i_0, p, i_2, (b+i_2))⋅U(i_3, i_1, (b+k))'))⋅4.0))
yesdirected!

  0.000473 seconds (3.29 k allocations: 146.906 KiB)
3 sum_out
∑((k, q, i_3), (U(q, i_1, k)⋅∑((p, b), (U(p, i_1, (b+i_2))⋅S(i_3, q, (b+k), k)⋅S(i_0, p, i_2, (b+i_2))⋅U(i_3, i_1, (b+k))'))⋅4.0))
∑((q, i_3), (∑((k), (U(q, i_1, k)⋅∑((p, b), (U(p, i_1, (b+i_2))⋅S(i_3, q, (b+k), k)⋅S(i_0, p, i_2, (b+i_2))⋅U(i_3, i_1, (b+k))'))))⋅4.0))
yesdir

(i_0, i_1, i_2) -> 
    (∑((q, k), (U(q, i_1, k)⋅∑((b), (∑((i_3), (S(i_3, q, (b+k), k)⋅U(i_3, i_1, (b+k))'))⋅∑((p), (U(p, i_1, (b+i_2))⋅S(i_0, p, i_2, (b+i_2))))))))⋅4.0)

In [36]:
pprof()

┌ Warning: The profile data buffer is full; profiling probably terminated
│ before your program finished. To profile for longer runs, call
│ `Profile.init()` with a larger buffer and/or larger delay.
└ @ Profile /Applications/Julia-1.9.app/Contents/Resources/julia/share/julia/stdlib/v1.9/Profile/src/Profile.jl:639


"profile.pb.gz"

Main binary filename not available.
Serving web UI on http://localhost:57599


In [13]:
settings = Dict(:symmetry => true)
nb(n) = neighbors(n; settings=settings)
fc(g_3)

(2.0⋅(∑((b, q), (∑((k, q), (U(q, i_1, (b+k))'⋅∑((p), (S(p, q, k, (b+k))'⋅U(p, i_1, k)))))⋅U(q, i_1, (i_2+b))⋅S(i_0, q, i_2, (i_2+b))))+∑((b, p), (S(p, i_0, (i_2+(-1.0⋅b)), i_2)'⋅∑((k, q), (∑((p), (U(p, i_1, k)'⋅S(p, q, k, (b+k))))⋅U(q, i_1, (b+k))))⋅U(p, i_1, (i_2+(-1.0⋅b)))))))

In [19]:
@profview fc(g_3) |> nb

Gtk.GtkWindowLeaf(name="", parent, width-request=-1, height-request=-1, visible=TRUE, sensitive=TRUE, app-paintable=FALSE, can-focus=FALSE, has-focus=FALSE, is-focus=FALSE, focus-on-click=TRUE, can-default=FALSE, has-default=FALSE, receives-default=FALSE, composite-child=FALSE, style, events=0, no-show-all=FALSE, has-tooltip=FALSE, tooltip-markup=NULL, tooltip-text=NULL, window, opacity=1.000000, double-buffered, halign=GTK_ALIGN_FILL, valign=GTK_ALIGN_FILL, margin-left, margin-right, margin-start=0, margin-end=0, margin-top=0, margin-bottom=0, margin=0, hexpand=FALSE, vexpand=FALSE, hexpand-set=FALSE, vexpand-set=FALSE, expand=FALSE, scale-factor=1, border-width=0, resize-mode, child, type=GTK_WINDOW_TOPLEVEL, title="Profile  -  11:21:24", role=NULL, resizable=TRUE, modal=FALSE, window-position=GTK_WIN_POS_NONE, default-width=800, default-height=600, destroy-with-parent=FALSE, hide-titlebar-when-maximized=FALSE, icon, icon-name=NULL, screen, type-hint=GDK_WINDOW_TYPE_HINT_NORMAL, skip

In [70]:
y = ((nb(fc(g_3))[2] |> first |> nb)[1] |> first |> nb)[4] |> first

((∑((b, q), (U(q, i_1, (i_2+(b⋅-1.0)))⋅S(q, i_0, (i_2+(b⋅-1.0)), i_2)'⋅∑((k, q), (U(q, i_1, (k+(b⋅-1.0)))'⋅∑((p), (U(p, i_1, k)⋅S(q, p, (k+(b⋅-1.0)), k)))))))+∑((b, p), (U(p, i_1, (i_2+(b⋅-1.0)))⋅S(p, i_0, (i_2+(b⋅-1.0)), i_2)'⋅∑((k, q), (U(q, i_1, (b+k))⋅∑((p), (S(p, q, k, (b+k))⋅U(p, i_1, k)')))))))⋅2.0)

In [92]:
@profile (y |> nb)
pprof()

┌ Warning: The profile data buffer is full; profiling probably terminated
│ before your program finished. To profile for longer runs, call
│ `Profile.init()` with a larger buffer and/or larger delay.
└ @ Profile /Applications/Julia-1.9.app/Contents/Resources/julia/share/julia/stdlib/v1.9/Profile/src/Profile.jl:639


"profile.pb.gz"

Main binary filename not available.
Serving web UI on http://localhost:57599
no matches found for regexp: 


In [29]:
a, b = content(fc(last(fc(fc(g_3)))))
b_s = simplify(b, settings=Dict(:symmetry=>true))

  0.005184 seconds (20.85 k allocations: 768.937 KiB, 56.17% compilation time: 100% of which was recompilation)
1 sum_in
∑((b, p), (U(p, i_1, ((-1.0⋅b)+i_2))⋅∑((k, q), (U(q, i_1, (b+k))⋅∑((p), (S(p, q, k, (b+k))⋅U(p, i_1, k)'))))⋅S(p, i_0, ((-1.0⋅b)+i_2), i_2)'))
∑((b, p, k, q), (U(q, i_1, (b+k))⋅U(p, i_1, ((-1.0⋅b)+i_2))⋅∑((p), (S(p, q, k, (b+k))⋅U(p, i_1, k)'))⋅S(p, i_0, ((-1.0⋅b)+i_2), i_2)'))
yesdirected!

  0.002427 seconds (26.96 k allocations: 941.922 KiB)
2 sum_in
∑((b, p, k, q), (U(q, i_1, (b+k))⋅U(p, i_1, ((-1.0⋅b)+i_2))⋅∑((p), (S(p, q, k, (b+k))⋅U(p, i_1, k)'))⋅S(p, i_0, ((-1.0⋅b)+i_2), i_2)'))
∑((b, p, k, q, i_3), (U(q, i_1, (b+k))⋅U(p, i_1, ((-1.0⋅b)+i_2))⋅S(i_3, q, k, (b+k))⋅U(i_3, i_1, k)'⋅S(p, i_0, ((-1.0⋅b)+i_2), i_2)'))
yesdirected!

  0.002043 seconds (27.14 k allocations: 942.297 KiB)
3 shift
∑((b, p, k, q, i_3), (U(q, i_1, (b+k))⋅U(p, i_1, ((-1.0⋅b)+i_2))⋅S(i_3, q, k, (b+k))⋅U(i_3, i_1, k)'⋅S(p, i_0, ((-1.0⋅b)+i_2), i_2)'))
∑((b, p, k, q, i_3), (U(q, i_1, k)⋅U(p, i

  0.049465 seconds (532.39 k allocations: 16.307 MiB, 15.62% gc time)
24 symmetry
∑((b, p, k, q, i_3), (U(q, i_1, k)⋅U(p, i_1, (b+i_2))⋅U(i_3, i_1, (b+k))'⋅S(q, i_3, k, (b+k))'⋅S(p, i_0, (b+i_2), i_2)'))
∑((b, p, k, q, i_3), (U(q, i_1, k)⋅U(p, i_1, (b+i_2))⋅S(i_0, p, i_2, (b+i_2))⋅U(i_3, i_1, (b+k))'⋅S(q, i_3, k, (b+k))'))
notdirected

  0.033354 seconds (447.12 k allocations: 13.405 MiB)
25 sum_sym
∑((b, p, k, q, i_3), (U(q, i_1, ((-1.0⋅b)+k))⋅U(p, i_1, (b+i_2))⋅U(i_3, i_1, k)'⋅S(q, i_3, ((-1.0⋅b)+k), k)'⋅S(p, i_0, (b+i_2), i_2)'))
∑((b, p, k, q, i_3), (U(q, i_1, (b+k))⋅U(p, i_1, ((-1.0⋅b)+i_2))⋅U(i_3, i_1, k)'⋅S(q, i_3, (b+k), k)'⋅S(p, i_0, ((-1.0⋅b)+i_2), i_2)'))
notdirected

  0.050764 seconds (537.40 k allocations: 16.447 MiB, 16.11% gc time)
26 symmetry
∑((b, p, k, q, i_3), (U(q, i_1, ((-1.0⋅b)+k))⋅U(p, i_1, (b+i_2))⋅U(i_3, i_1, k)'⋅S(q, i_3, ((-1.0⋅b)+k), k)'⋅S(p, i_0, (b+i_2), i_2)'))
∑((b, p, k, q, i_3), (U(q, i_1, ((-1.0⋅b)+k))⋅U(p, i_1, (b+i_2))⋅S(i_0, p, i_2, (b+i_2))⋅U(i_3

3-element Vector{PCT.AbstractPCTNode}:
 ∑((b, p, k, q, i_3), (U(q, i_1, k)⋅U(p, i_1, (b+i_2))⋅S(i_3, q, (b+k), k)⋅S(i_0, p, i_2, (b+i_2))⋅U(i_3, i_1, (b+k))'))
 ∑((b, p, k, q, i_3), (U(q, i_1, k)⋅U(p, i_1, (b+i_2))⋅S(i_3, q, (b+k), k)⋅S(i_0, p, i_2, (b+i_2))⋅U(i_3, i_1, (b+k))'))
 ∑((b, p, k, q, i_3), (U(q, i_1, k)⋅U(p, i_1, (b+i_2))⋅S(i_3, q, (b+k), k)⋅S(i_0, p, i_2, (b+i_2))⋅U(i_3, i_1, (b+k))'))

In [30]:
a_s = simplify(a, settings=Dict(:symmetry=>true))

  0.002015 seconds (25.23 k allocations: 842.328 KiB)
1 sum_in
∑((b, q), (U(q, i_1, (b+i_2))⋅S(i_0, q, i_2, (b+i_2))⋅∑((k, q), (∑((p), (U(p, i_1, k)⋅S(p, q, k, (b+k))'))⋅U(q, i_1, (b+k))'))))
∑((b, q, k, i_3), (U(q, i_1, (b+i_2))⋅S(i_0, q, i_2, (b+i_2))⋅∑((p), (U(p, i_1, k)⋅S(p, i_3, k, (b+k))'))⋅U(i_3, i_1, (b+k))'))
yesdirected!

  0.002503 seconds (37.10 k allocations: 1.181 MiB)
2 sum_in
∑((b, q, k, i_3), (U(q, i_1, (b+i_2))⋅S(i_0, q, i_2, (b+i_2))⋅∑((p), (U(p, i_1, k)⋅S(p, i_3, k, (b+k))'))⋅U(i_3, i_1, (b+k))'))
∑((b, q, k, i_3, p), (U(q, i_1, (b+i_2))⋅U(p, i_1, k)⋅S(i_0, q, i_2, (b+i_2))⋅U(i_3, i_1, (b+k))'⋅S(p, i_3, k, (b+k))'))
yesdirected!

  0.001774 seconds (24.92 k allocations: 855.938 KiB)
3 shift
∑((b, q, k, i_3, p), (U(q, i_1, (b+i_2))⋅U(p, i_1, k)⋅S(i_0, q, i_2, (b+i_2))⋅U(i_3, i_1, (b+k))'⋅S(p, i_3, k, (b+k))'))
∑((b, q, k, i_3, p), (U(q, i_1, (b+i_2))⋅U(p, i_1, ((-1.0⋅b)+k))⋅S(i_0, q, i_2, (b+i_2))⋅U(i_3, i_1, k)'⋅S(p, i_3, ((-1.0⋅b)+k), k)'))
notdirected

  0.006161 

  0.040655 seconds (524.94 k allocations: 16.106 MiB)
25 sum_sym
∑((b, q, k, i_3, p), (U(q, i_1, ((-1.0⋅b)+i_2))⋅U(p, i_1, k)⋅U(i_3, i_1, ((-1.0⋅b)+k))'⋅S(q, i_0, ((-1.0⋅b)+i_2), i_2)'⋅S(p, i_3, k, ((-1.0⋅b)+k))'))
∑((b, q, k, i_3, p), (U(q, i_1, (b+i_2))⋅U(p, i_1, k)⋅U(i_3, i_1, (b+k))'⋅S(q, i_0, (b+i_2), i_2)'⋅S(p, i_3, k, (b+k))'))
notdirected

  0.042177 seconds (454.30 k allocations: 13.575 MiB, 18.75% gc time)
26 symmetry
∑((b, q, k, i_3, p), (U(q, i_1, ((-1.0⋅b)+i_2))⋅U(p, i_1, k)⋅U(i_3, i_1, ((-1.0⋅b)+k))'⋅S(q, i_0, ((-1.0⋅b)+i_2), i_2)'⋅S(p, i_3, k, ((-1.0⋅b)+k))'))
∑((b, q, k, i_3, p), (U(q, i_1, ((-1.0⋅b)+i_2))⋅U(p, i_1, k)⋅S(i_3, p, ((-1.0⋅b)+k), k)⋅U(i_3, i_1, ((-1.0⋅b)+k))'⋅S(q, i_0, ((-1.0⋅b)+i_2), i_2)'))
notdirected

  0.038385 seconds (492.97 k allocations: 15.265 MiB)
27 symmetry
∑((b, q, k, i_3, p), (U(q, i_1, ((-1.0⋅b)+i_2))⋅U(p, i_1, k)⋅S(i_0, q, i_2, ((-1.0⋅b)+i_2))⋅U(i_3, i_1, ((-1.0⋅b)+k))'⋅S(p, i_3, k, ((-1.0⋅b)+k))'))
∑((b, q, k, i_3, p), (U(q, i_1, ((-1.0⋅b)

4-element Vector{PCT.AbstractPCTNode}:
 ∑((b, q, k, i_3, p), (U(q, i_1, (b+i_2))⋅U(p, i_1, k)⋅S(i_3, p, (b+k), k)⋅S(i_0, q, i_2, (b+i_2))⋅U(i_3, i_1, (b+k))'))
 ∑((b, q, k, i_3, p), (U(q, i_1, (b+i_2))⋅U(p, i_1, k)⋅S(i_3, p, (b+k), k)⋅S(i_0, q, i_2, (b+i_2))⋅U(i_3, i_1, (b+k))'))
 ∑((b, q, k, i_3, p), (U(q, i_1, (b+i_2))⋅U(p, i_1, k)⋅S(i_3, p, (b+k), k)⋅S(i_0, q, i_2, (b+i_2))⋅U(i_3, i_1, (b+k))'))
 ∑((b, q, k, i_3, p), (U(q, i_1, (b+i_2))⋅U(p, i_1, k)⋅S(i_3, p, (b+k), k)⋅S(i_0, q, i_2, (b+i_2))⋅U(i_3, i_1, (b+k))'))

In [63]:
simplify(add(a_s |> first, b_s |> first)) |> first

  0.004051 seconds (2.15 k allocations: 141.438 KiB, 99.19% compilation time: 100% of which was recompilation)
1 sum_in
(∑((b, p, k, q, i_3), (U(q, i_1, k)⋅U(p, i_1, (b+i_2))⋅S(i_3, q, (b+k), k)⋅S(i_0, p, i_2, (b+i_2))⋅U(i_3, i_1, (b+k))'))⋅2.0)
∑((b, p, k, q, i_3), (U(q, i_1, k)⋅U(p, i_1, (b+i_2))⋅S(i_3, q, (b+k), k)⋅S(i_0, p, i_2, (b+i_2))⋅2.0⋅U(i_3, i_1, (b+k))'))
yesdirected!

  0.000000 seconds


∑((b, p, k, q, i_3), (U(q, i_1, k)⋅U(p, i_1, (b+i_2))⋅S(i_3, q, (b+k), k)⋅S(i_0, p, i_2, (b+i_2))⋅2.0⋅U(i_3, i_1, (b+k))'))

In [34]:
aa = a_s |> first

∑((b, q, k, i_3, p), (U(q, i_1, (b+i_2))⋅U(p, i_1, k)⋅S(i_3, p, (b+k), k)⋅S(i_0, q, i_2, (b+i_2))⋅U(i_3, i_1, (b+k))'))

In [35]:
bb = b_s |> first

∑((b, p, k, q, i_3), (U(q, i_1, k)⋅U(p, i_1, (b+i_2))⋅S(i_3, q, (b+k), k)⋅S(i_0, p, i_2, (b+i_2))⋅U(i_3, i_1, (b+k))'))

In [62]:
aa == bb

true

In [57]:
aa.signatures[5] == bb.signatures[4]

true

In [40]:
bb.signatures[1]

: Mul 
    1: PCTVector 
        1: PrimitiveCall 
            1: Var{MapType} U
            3: Var{I} i_1
        2: PrimitiveCall 
            1: Var{MapType} U
            3: Var{I} i_1
            4: Add 
                1: PCTVector 
                    1: Var{Domain} 
                    2: Var{I} i_2
        3: PrimitiveCall 
            1: Var{MapType} S
            4: Add 
                1: PCTVector 
                    1: Var{Domain} 
        4: PrimitiveCall 
            1: Var{MapType} S
            2: Var{I} i_0
            4: Var{I} i_2
            5: Add 
                1: PCTVector 
                    1: Var{Domain} 
                    2: Var{I} i_2
        5: Conjugate 
            1: PrimitiveCall 
                1: Var{MapType} U
                3: Var{I} i_1
                4: Add 
                    1: PCTVector 
                        1: Var{Domain} 


In [42]:
aa.signatures[1]

: Mul 
    1: PCTVector 
        1: PrimitiveCall 
            1: Var{MapType} U
            3: Var{I} i_1
            4: Add 
                1: PCTVector 
                    1: Var{Domain} 
                    2: Var{I} i_2
        2: PrimitiveCall 
            1: Var{MapType} U
            3: Var{I} i_1
        3: PrimitiveCall 
            1: Var{MapType} S
            4: Add 
                1: PCTVector 
                    1: Var{Domain} 
        4: PrimitiveCall 
            1: Var{MapType} S
            2: Var{I} i_0
            4: Var{I} i_2
            5: Add 
                1: PCTVector 
                    1: Var{Domain} 
                    2: Var{I} i_2
        5: Conjugate 
            1: PrimitiveCall 
                1: Var{MapType} U
                3: Var{I} i_1
                4: Add 
                    1: PCTVector 
                        1: Var{Domain} 
